In [1]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=af076ce7a8daee6dcb20991dcdf6b04371f071552a68d8fbeb9c15c334996010
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from fpdf import FPDF
import tempfile
import os

plt.switch_backend('agg')

china = gpd.read_file("/content/drive/MyDrive/Data/China.json")

config = {
    "colors": {
        "High-High": "#4daf4a",
        "Low-Low": "#a6d854",
        "Negative": "#e78ac3",
        "Low-High": "#ffd92f",
        "Not Significant": "#e0e0e0"
    },
    "years": {
        1991: {"Low-Low": ["Sichuan", "Yunnan"]},
        2005: {"High-High": ["Zhejiang"], "Low-Low": ["Sichuan", "Yunnan"], "Negative": ["Anhui"]},
        2009: {"High-High": ["Zhejiang"], "Low-Low": ["Yunnan", "Sichuan", "Hubei", "Chongqing"], "Negative": ["Anhui"]},
        2010: {"High-High": ["Zhejiang"], "Low-High": ["Anhui"]}
    }
}

def create_figure():
    fig = plt.figure(figsize=(14, 12))
    gs = fig.add_gridspec(3, 2, height_ratios=[1, 1, 0.2])

    axes = [
        fig.add_subplot(gs[0, 0]),
        fig.add_subplot(gs[0, 1]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[1, 1])
    ]
    legend_ax = fig.add_subplot(gs[2, :])

    legend_elements = [
        Patch(facecolor="#4daf4a", label="High-High Cluster"),
        Patch(facecolor="#a6d854", label="Low-Low Cluster"),
        Patch(facecolor="#e78ac3", label="Negative Outlier"),
        Patch(facecolor="#ffd92f", label="Low-High Outlier"),
        Patch(facecolor="#e0e0e0", label="Not Significant")
    ]

    for idx, year in enumerate(config["years"]):
        ax = axes[idx]
        ax.set_title(f"{year}", fontsize=12, pad=10)
        ax.set_axis_off()
        china.plot(ax=ax, color="#e0e0e0", edgecolor="black", linewidth=0.3)

        for cluster, provinces in config["years"][year].items():
            for prov in provinces:
                if prov in china["NAME_1"].values:
                    china[china["NAME_1"] == prov].plot(
                        ax=ax,
                        color=config["colors"].get(cluster, "#e0e0e0"),
                        edgecolor="black",
                        linewidth=0.3
                    )

    legend_ax.axis('off')
    legend = legend_ax.legend(
        handles=legend_elements,
        loc='center',
        ncol=5,
        frameon=False,
        fontsize=10,
        title="LISA Cluster Classification",
        title_fontsize=11
    )
    plt.setp(legend.get_title(), fontweight='bold')

    plt.tight_layout()
    return fig

try:
    fig = create_figure()
    temp_img = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    plt.savefig(temp_img.name, dpi=300, bbox_inches='tight')
    plt.close()

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.ln(5)
    pdf.image(temp_img.name, x=10, y=30, w=190)

    output_path = "/content/drive/MyDrive/Data/china.pdf"
    pdf.output(output_path)

    temp_img.close()
    os.unlink(temp_img.name)
    print(f"PDF successfully saved to: {output_path}")

except Exception as e:
    print(f"Error: {str(e)}")


PDF successfully saved to: /content/drive/MyDrive/Data/china.pdf
